# Example 1.1
This example follows the `examples/1.1`  
Search for a simple lj crystal with 8 numbers percell

In [1]:
from airsspy import SeedAtoms, Buildcell
from tqdm.auto import tqdm

# Import ase assets
from ase.optimize import BFGSLineSearch, sciopt, FIRE, BFGS, precon
from ase.calculators.lj import LennardJones
from ase.constraints import UnitCellFilter, ExpCellFilter

In [2]:
# Define the search seed

seed = SeedAtoms('Al', cell=[2, 2, 2], pbc=True)
seed.gentags.minsep = 1.5
# Define per-atom tags, here it is simply request 8 Al atoms
al = seed[0]
al.num = 8

In [3]:
# LJ potential calculator
lj = LennardJones(signma=2, epsilon=1, beta=1, rc=2.5)

In [4]:
# The seed in text form
print('\n'.join(seed.get_cell_inp_lines()))

%BLOCK lattice_cart
2.0000000000  0.0000000000  0.0000000000
0.0000000000  2.0000000000  0.0000000000
0.0000000000  0.0000000000  2.0000000000
%ENDBLOCK lattice_cart
%BLOCK positions_abs
Al  0.0000000000 0.0000000000 0.0000000000 # Al0 % NUM=8
%ENDBLOCK positions_abs
#MINSEP=1.5



## Generate the random structure
The random structrue can be built by calling `build_random_atoms` method.
An `ase.Atoms` object is returned.

In [5]:
rand_atoms = seed.build_random_atoms()

Inpsect the cell

In [6]:
rand_atoms.get_cell()

Cell([[2.12005, 0.0, 0.0], [-0.7258250466176692, 4.554031453756381, 0.0], [-0.6083780555145146, -1.7092519801209565, 6.33733973642892]])

Inspect the cell volume
The initial cell volume is $2\times2\times2=8$ and we requested 8 atoms.
So the target cell volume is 64 but `buildcell` will scale to a random value
within 50%.

In [7]:
rand_atoms.get_volume()

61.1855853470401

In [8]:
rand_atoms.get_positions()

array([[1.33016898, 0.54343656, 5.45596852],
       [1.07291236, 0.25251317, 3.49864838],
       [0.2228569 , 1.66269419, 3.35633869],
       [1.50858009, 1.47088937, 0.05854071],
       [2.0562808 , 2.43838309, 3.00431118],
       [0.15568396, 0.70533803, 4.79180677],
       [2.02194829, 1.84321829, 4.68757286],
       [0.66264855, 2.83948195, 0.49940956]])

## Do the search
The search can be carried out directly using the calculator and optimizers in `ase` 

In [9]:
def gen_and_relax(seed, calc, opti):
    """
    Generate and relax the cell
    """
    rand_atoms = seed.build_random_atoms()
    rand_atoms.set_pbc(True)  # Set the PBC as we are search from crystal
    rand_atoms.set_calculator(calc)  # Attached hte calculator
    # Apply the UnitCellFilter as we are also optimizing the cell
    opt = opti(UnitCellFilter(rand_atoms), logfile=None) 
    opt.run(fmax=0.05)
    return rand_atoms

In [ ]:
res = []
for i in tqdm(range(20)):
    # A range of the possible optimization algorithm are possible
    relaxed = gen_and_relax(seed, lj, BFGSLineSearch)
    res.append(relaxed)

## Get the symmetry of the relaxed structures

In [12]:
from spglib import get_spacegroup
spg = [get_spacegroup(i, 0.5) for i in res]
spg

['Fm-3m (225)',
 'Fm-3m (225)',
 'Fm-3m (225)',
 'P2_1/m (11)',
 'Fm-3m (225)',
 'Cm (8)',
 'P6_3/mmc (194)',
 'Fm-3m (225)',
 'P6_3/mmc (194)',
 'C2/m (12)',
 'Fm-3m (225)',
 'Fm-3m (225)',
 'Fm-3m (225)',
 'Imma (74)',
 'Cm (8)',
 'C2/m (12)',
 'Fm-3m (225)',
 'P6_3/mmc (194)',
 'P-1 (2)',
 'P6_3/mmc (194)']

## Alternatively, use the native airss script to run the search

In [11]:
!mkdir work

In [15]:
# The content of the LJ potential file
pp = "1 12 6 2.5\nAl\n# Epsilon\n1\n# Sigma\n2\n# Beta\n1"

In [17]:
%%timeit -n 1 -r 1
seed.write_seed('work/Al.cell')
from pathlib import Path
Path("work/Al.pp").write_text(pp)
!cd work && airss.pl -seed Al -max 20 -pp3 && cd ../

7.44 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


Note that the `pp3` program for relxation will run significantly faster than doing so in python

In [18]:
!cd work/ &&  ca -r -u 0.01 && cd -

Al-581-4050-17          -0.00     7.561      -6.659   8 Al           P63/mmc    1
Al-581-4050-4           -0.00     7.561       0.000   8 Al           P63/mmc    1
Al-581-4050-6           -0.00     7.561       0.000   8 Al           P63/mmc    1
Al-581-4050-19          -0.00     7.561       0.000   8 Al           P63/mmc    1
Al-581-4050-18          -0.00     7.562       0.002   8 Al           R-3m       1
Al-581-4050-9            0.00     7.562       0.002   8 Al           R-3m       1
Al-581-4050-11          -0.00     7.564       0.005   8 Al           Fm-3m      1
Al-581-4050-10          -0.00     7.564       0.005   8 Al           Fm-3m      1
Al-581-4050-2            0.00     7.564       0.005   8 Al           Fm-3m      1
Al-581-4050-14           0.00     7.564       0.005   8 Al           Fm-3m      1
Al-581-4050-20          -0.00     7.564       0.005   8 Al           Fm-3m      1
Al-581-4050-8            0.00     7.564       0.005   8 Al           Fm-3m      1
Al-581-4050-3   